In [82]:
import ee
import geemap
import math 
import json
import pandas as pd

In [83]:
from utils.utils import *
from utils.ee_utils import *
from utils.utils import TextColors as c

In [84]:
Map = geemap.Map()

In [85]:
ee.Initialize()

In [86]:
with open("shapefiles\Europe.geojson") as f: geojson = json.load(f)
ee_object = geemap.geojson_to_ee(geojson)

In [87]:
Map.addLayer(ee_object, {}, 'Europe')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [88]:
randomPoints = ee.FeatureCollection.randomPoints(ee_object, 1000, 0)

In [89]:
randomPoints_json = randomPoints.getInfo()

In [90]:
randomPoints_json['features'][0]['geometry']['coordinates']

[15.199206788344531, 51.08562704162501]

In [91]:
# create an empty dataframe with the desired columns
df = pd.DataFrame(columns=['Point_ID', 'long', 'lat','OC'])

# loop through the features of the randomPoints object to populate the dataframe
for i, point in enumerate(randomPoints_json['features']):
    lon = point['geometry']['coordinates'][0]
    lat = point['geometry']['coordinates'][1]
    df.loc[i] = [i, lon, lat, 0]

# convert point_id column to integer type
df['Point_ID'] = df['Point_ID'].astype(int)

# save the dataframe as a csv file
df.to_csv('randomPoints.csv', index=True, index_label='OID_')


In [92]:
random_point = df.sample(n=1)


In [93]:
random_point

,Point_ID,long,lat,OC
705,705,38.514727,54.813319,0.0


In [94]:
lat = random_point['lat'].values[0]
lon = random_point['long'].values[0]

In [95]:
roi = get_square_roi(lat,lon, roi_size=10000, return_gee_object=True)
Map.addLayer(roi, {}, 'ROI')
Map.centerObject(roi, 10)


In [96]:
land_cover = ee.ImageCollection('ESA/WorldCover/v100').first().clip(roi)
vis = {'bands': ['Map']}
Map.addLayer(land_cover, vis, 'Land Cover')


In [97]:
def calculate_percentage(image, values,scale=30, roi=roi):
    """
    Calculates the percentage of pixels in the given image that have the specified values within the ROI.

    Args:
    image (ee.Image): The image to calculate the percentage for.
    values (list): A list of values to calculate the percentage for.
    scale (int, optional): The scale of the reduction. Defaults to 30. The larger the scale, the faster the calculation, but the less accurate it is.
    roi (ee.Geometry, optional): The region of interest. Defaults to roi.

    Returns:
    float: The percentage of pixels in the image that have the specified values within the ROI.
    """
    # calculate the total number of pixels
    total_pixels = image.select('Map').reduceRegion(reducer=ee.Reducer.count(), geometry=roi, scale=scale).get('Map')
    value_pixels = ee.Number(0)
    for value in values:   
        value_p =  ee.Number(image.select('Map').eq(value).reduceRegion(reducer=ee.Reducer.sum(), geometry=roi, scale=scale).get('Map'))
        value_pixels = value_pixels.add(value_p)
    
    # calculate the percentage of pixels with the given value
    percentage = value_pixels.divide(total_pixels).multiply(100)
    
    return percentage


In [98]:
percentage = calculate_percentage(land_cover, [80,90,50,100,95], scale=30, roi=roi).getInfo()
print(f"The percentage of pixels with value  is {percentage}%")

The percentage of pixels with value  is 5.831409432308125%


In [100]:
# add unwanted cover column to the dataframe
df['unwanted_cover'] = 0
for i,point in enumerate(df.iterrows()):
    
    lat = point[1]['lat']
    lon = point[1]['long']
    roi = get_square_roi(lat,lon, roi_size=1920, return_gee_object=True)
    land_cover = ee.ImageCollection('ESA/WorldCover/v100').first().clip(roi)
    percentage = calculate_percentage(land_cover, [80,90,50,100,95], scale=30, roi=roi).getInfo()
    df.loc[point[0], 'unwanted_cover'] = percentage
    if i%10 == 0:
        print(f"Processed {i} points")

Processed 0 points
Processed 10 points
Processed 20 points
Processed 30 points
Processed 40 points
Processed 50 points
Processed 60 points
Processed 70 points
Processed 80 points
Processed 90 points
Processed 100 points
Processed 110 points
Processed 120 points
Processed 130 points
Processed 140 points


KeyboardInterrupt: 

In [ ]:
# save the dataframe as a csv file
df.to_csv('randomPoints.csv', index=True, index_label='OID_')